<a href="https://colab.research.google.com/github/arifmhassan/Spark-Python-Data-Ingestion/blob/master/RetailSales_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
ta

#Aggregations
Primary Data source=online-retail-dataset.csv
purpose: Aggregation using groupby, windows, rollup, sql like dense ranking

Question
*   dataframe read  the data online-retail-dataset.csv with 5 partition
*   Data analysis count distinct e.g. stockcode, min quantity, max quantity, sum of quantity, sum of revenue by Country, CUstomer, date, hour  etc

*   Whick date has the maximum  purchase quantity using dense_rank in sql and python
*   group by accross country/date/ quantity using rollup and cube
*    group by accross country/date/ quantity using pivot to get country listing in a column



In [1]:
# 

sample_data/
